In [22]:
import rosbag2_py
from rclpy.serialization import serialize_message
import std_msgs.msg as std_msgs
import numpy as np
from builtin_interfaces.msg import Time
import ros2_numpy as rnpy

In [23]:
from waymo_open_dataset import v2
import matplotlib.pyplot as plt
import tensorflow as tf
import dask.dataframe as dd

import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)

In [24]:
# Path to the directory with all components
dataset_dir = '/home/myron/data/Waymo/data'
context = '10504764403039842352_460_000_480_000'

lidar_df = dd.read_parquet(f'{dataset_dir}/lidar/testing_lidar_{context}.parquet')
lidar_calib_df = dd.read_parquet(f'{dataset_dir}/lidar_calibration/testing_lidar_calibration_{context}.parquet')
lidar_pose_df = dd.read_parquet(f'{dataset_dir}/lidar_pose/testing_lidar_pose_{context}.parquet')
# vehicle_pose_df = dd.read_parquet(f'{dataset_dir}/vehicle_pose/testing_vehicle_pose_10504764403039842352_460_000_480_000.parquet')

In [25]:
lidar_lcalib_df = v2.merge(lidar_df, lidar_calib_df, right_group=True)

In [26]:
lidar_lcalib_df.head()

,key.segment_context_name,key.frame_timestamp_micros,key.laser_name,[LiDARComponent].range_image_return1.values,[LiDARComponent].range_image_return1.shape,[LiDARComponent].range_image_return2.values,[LiDARComponent].range_image_return2.shape,[LiDARCalibrationComponent].extrinsic.transform,[LiDARCalibrationComponent].beam_inclination.min,[LiDARCalibrationComponent].beam_inclination.max,[LiDARCalibrationComponent].beam_inclination.values
0,10504764403039842352_460_000_480_000,1558060923374527,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
1,10504764403039842352_460_000_480_000,1558060923474551,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
2,10504764403039842352_460_000_480_000,1558060923574609,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
3,10504764403039842352_460_000_480_000,1558060923674630,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
4,10504764403039842352_460_000_480_000,1558060923774688,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."


In [27]:
class RosbagWriter:
    def __init__(self, topic_msg_type_map: dict, output_bag_path: str, serialization='cdr') -> None:
        self.writer = rosbag2_py.SequentialWriter()
        self.open_bag_for_writing(output_bag_path, serialization)
        for topic, msg_type in topic_msg_type_map.items():
            self.writer.create_topic(rosbag2_py._storage.TopicMetadata(
                name=topic,
                type=msg_type,
                serialization_format=serialization))
    
    def open_bag_for_writing(self, path, serialization_format='cdr'):
        storage_options = rosbag2_py.StorageOptions(uri=path, storage_id='sqlite3')

        converter_options = rosbag2_py.ConverterOptions(
            input_serialization_format=serialization_format,
            output_serialization_format=serialization_format)

        self.writer.open(storage_options, converter_options)
    
    def write(self, topic: str, ros2_msg: any, timestamp_ns: int):
        self.writer.write(topic, serialize_message(ros2_msg), timestamp_ns)

In [28]:
bag_out_path = f'output/waymo_{context}'
rb_writer = RosbagWriter({'/points2': 'sensor_msgs/msg/PointCloud2'}, bag_out_path)

[INFO] [1680087855.650289736] [rosbag2_storage]: Opened database 'output/waymo_10504764403039842352_460_000_480_000/waymo_10504764403039842352_460_000_480_000_0.db3' for READ_WRITE.


In [29]:
def lidar_range_img_to_pcd_xyzi_msg(range_img, lidar_calib, lidar_frame: str, timestamp_us: int):
    pc = v2.convert_range_image_to_point_cloud(range_img, lidar_calib, keep_polar_features=True)
    ts = Time(sec=int(timestamp_us/1e6), nanosec=int((timestamp_us%1e6)*1e3))
    pc = np.rec.fromarrays(tf.transpose(tf.gather(pc, [3,4,5,1], axis=1)),
                           dtype={'names': ('x', 'y', 'z', 'i'),
                                  'formats': (np.float32, np.float32, np.float32, np.float32)})
    return rnpy.array_to_pointcloud2(pc, ts, lidar_frame)

In [30]:
for i, (_, r) in enumerate(lidar_lcalib_df.iterrows()):
  # Create component dataclasses for the raw data
  lidar = v2.LiDARComponent.from_dict(r)
  timestamp_us = lidar.key.frame_timestamp_micros
  # lidar_pose = v2.LiDARPoseComponent.from_dict(r)
  lidar_calib = v2.LiDARCalibrationComponent.from_dict(r)
  laser_name = int(lidar.key.laser_name)
  # vehicle_pose = v2.VehiclePoseComponent.from_dict(r)

  if(laser_name == 1):
    pcd_msg = lidar_range_img_to_pcd_xyzi_msg(lidar.range_image_return1, lidar_calib, '/lidar', timestamp_us)
    rb_writer.write('/points2', pcd_msg, int(timestamp_us*1e3))

  # print(pc)
  # print(pc.shape)
  # if(i > 3):
  # break

In [31]:
del rb_writer